In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
torch.manual_seed(1)

x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [11]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


### 가설식, W,b 정의하기

시그모이드 함수는 

$$
\sigma(z) = \frac{1}{1+e^{-z}}
$$

이 함수를 사용하는 이유는,  이진분류에서  결국  나와야 하는 값은  0~1 사이의 확률이기 때문이다.  ( 이 함수의 모양은 0~1 사이를 매끄럽게 연결해주느 모양)

결국 우리가 할 일은,  주어진 이진분류데이터 셋
:  ( $x_i$, 0 또는 1)  
들을 학습시켜서
( 여기서 0~1은 강아지 사진일 확률,  데이터 $x_i$는 임의의 사진)

내가 어떤 값을 입력했을때, 그 사진이 강아지일 확률을 뽑아내는 알고리즘을 만들고 싶은것.

결과적으로, 

$$
\sigma( z = w^T X + b) = \frac{1}{1+e^{-(w^T X + b)}} = \text{강아지 사진일 확률}
$$

이 함수는 주어진 데이터, 파라미터에 대해서 0~1값을 주는 함수모델이된다.

이제, W랑 b를 적절히 찾아내어서,  내가 입력한 데이터 셋 ( $x_i$ , 0 or 1) 과 최대한 경향성이 유사하게 만들면된다.

### Cost function

내가 위에서 만든 모델함수와,  주어진 이진 training data set  이 최대한 비슷하게 만들고 싶다.

그러려면, cost function 을 정의해서  이거의 최소값을 구하면 된다.

다만, 이 경우에는 MSE 를 쓰면 안된다. local minima가 너무 많아서 그렇다. 대신해서 log fucntion을 이용하여 cost funciton을 작성한다.

$$
cost = -[y\,log\sigma(z) + (1-y)\,log(1-\sigma(z))]
$$

이렇게 cost를 지정하는 이유는,

1) $y=1$ 일때 (즉, 데이터 셋이 강아지 사진일때)  
$\sigma(z)$ 값이 1에 가까워 져야 == 강아지 사진일 확률이 커져야 cost가 작아진다.
손실 함수가 최소가 되는 지점 = 강아지 사진일때 $\sigma(z)$ 값이 1에 가까움.


2. $y=0$ 일때 (즉, 데이터 셋이 강아지 사진이 아님)
$\sigma(z)$ 값이 0에 가까워 져야 == 강아지가 아닐 확률이 커져야 cost가 작아진다.
손실 함수가 최소가 되는 지점 = 강아지 사진아닐때 $\sigma(z)$ 값이 0에 가까움.

In [12]:
# 2x1 벡터는 가설식 W^T * X + b 에서 W^T에 해당한다.
# 이진분류를 적절히 수행하기 위한 W와 b를 로지스틱 회귀를 통해서 찾아보자.


W = torch.zeros((2, 1), requires_grad=True) 
# requires_grad=True : 학습을 통해 계속 값이 변경되는 변수임을 명시
b = torch.zeros(1, requires_grad=True)

hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))
# or hypothesis = torch.sigmoid(x_train.matmul(W) + b)
# 데이터셋 행렬 x_train과 W의 행렬곱에 b를 더한 값에 시그모이드 함수를 적용한 값이 hypothesis이다.

print(hypothesis)


tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)


In [13]:
# 앞서 정의한 비용함수는 F로 정의한 클래스에 이미 구현되어 있다.
# F.binary_cross_entropy(예측값, 실제값)

Cost = F.binary_cross_entropy(hypothesis, y_train)

In [19]:
# optimizer 설정 ( 확률적 경사 하강법으로 최적화 방식 차용 )
# lr 은 learning rate으로, 한번의 gradient decent당 얼마만큼 이동할지를 설정한다.
optimizer = optim.SGD([W, b], lr=1)


# 에포크는 Cost를 최소화하기 위해서 몇번 최적화를 반복할 것인지를 명시.
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train).mean()

    # Reset gradients to zero
    # 각 반복마다, 기울기를 0으로 초기화해야한다. 왜냐하면, 이전 반복에서 구한 기울기값이 남아있기 때문이다. 
    # ( 기본 설정은 기울기를 반복문마다 누적해서 합산하도록 되어있음 )
    optimizer.zero_grad()

    # Compute gradients of the cost with respect to parameters W and b
    # cost함수는 W,b에 대한 함수이다. 따라서, cost를 W,b에 대해 미분하면 각각의 기울기를 구할 수 있다.
    # backward() 함수를 통해 역전파 알고리즘을 통해서 cost함수의 현재지점에서의 기울기를 구할 수 있다.
    cost.backward()

    # Update parameters W and b using the computed gradients
    # 이번 반복문에서 구한 기울기를 이용해서 W와 b를 업데이트한다.
    optimizer.step()


    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch    0/1000 Cost: 0.019834
Epoch  100/1000 Cost: 0.018149
Epoch  200/1000 Cost: 0.016730
Epoch  300/1000 Cost: 0.015517
Epoch  400/1000 Cost: 0.014469
Epoch  500/1000 Cost: 0.013554
Epoch  600/1000 Cost: 0.012748
Epoch  700/1000 Cost: 0.012033
Epoch  800/1000 Cost: 0.011394
Epoch  900/1000 Cost: 0.010819
Epoch 1000/1000 Cost: 0.010300


### Step-by-Step Explanation
Start of the Loop:

At the beginning of each iteration of the training loop, optimizer.zero_grad() is called to reset the gradients of all model parameters to zero.
This ensures that the gradients from the previous iteration do not interfere with the current iteration.


##### Forward Pass:

The input data is passed through the model to compute the predictions (hypothesis).
The loss (cost) is then calculated based on the predictions and the true labels.


##### Backward Pass:

The cost.backward() function is called to compute the gradients of the loss with respect to each model parameter.
These gradients are stored in the .grad attribute of each parameter.


##### Optimizer Step:

The optimizer.step() function is called to update the model parameters using the computed gradients.
The optimizer adjusts the parameters to minimize the loss.

##### Why Gradients Are Not Reset During Backward Pass<br><br>
Reset Before Backward Pass: The gradients are reset to zero before the backward pass using optimizer.zero_grad(). This ensures that the gradients computed during the backward pass are only based on the current iteration's loss.<br><br>
Accumulate During Backward Pass: During the backward pass, the gradients are computed and accumulated in the .grad attribute of each parameter. Since the gradients were reset to zero at the beginning of the iteration, they only reflect the current iteration's computations.<br><br>
Update After Backward Pass: After the backward pass, the optimizer updates the parameters using the accumulated gradients. The gradients are then reset again at the beginning of the next iteration.

### 학습 완료후, 추가 데이터 입력후 값 예측단계

In [15]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis)


tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward0>)


In [16]:
# 예측값이 0.5보다 크면 True, 아니면 False로 설정
# torch.FloatTensor 끼리의 크기비교는 각 원소별로 비교한 결과를 반환한다.
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction)


tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


### nn.Module / nn.Sequential 클래스 활용

nn.Sequential은 인수로 nn.Module 객체 ( 여기서는 nn.Linear와 nn.Sigmoid )를 받아서 순서대로 연결하여 신경망을 생성한다.

```python
class Sequential(nn.Module):
    def __init__(self, *args):
        super(Sequential, self).__init__()
        self.layers = nn.ModuleList(args)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
```

이렇게 nn.Sequential의 인수로 들어간 nn.Module의 객체는
nn.Sequential내부 에서 nn.ModuleList에 저장된다.

질문(1) 이렇게 층별로 쌓는 것은 어떤 이점이 존재하는가?

만약에 내가 만드는 모델의 레이어가 많아지고 복잡해지면 다루는게 상당히 까다로워진다. 왜냐면, 각 레이어층마다 forward 연산 ( linear , Activation function 등) 을 지정해주어야 하기 때문. => 디버깅 및 보기에 좀 불편하다.


##### nn.Sequential을 사용하지 않고 직접 만드는 경우
```python
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyNeuralNetwork, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5)
        self.fc1 = nn.Linear(in_features=30*5*5, out_features=128, bias=True)
        self.fc2 = nn.Linear(in_features=128, out_features=10, bias=True)


    # 이렇게 모든 레이어들에 대해서 한번에 forward 연산들을 지정해주어야 한다.
    def forward(self, x):

        # 레이어 1
        x = F.relu(self.conv1(x), inplace=True)
        x = F.max_pool2d(x, (2, 2))

        # 레이어2
        x = F.relu(self.conv2(x), inplace=True)
        x = F.max_pool2d(x, (2, 2))

        # 레이어3
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x), inplace=True)
        x = F.relu(self.fc2(x), inplace=True)

        return x
```


##### nn.Sequential 을 사용해서 좀더 가독성 높게 표현하는 방법
nn.Sequential is a convenient way to build neural networks by stacking layers sequentially. It allows you to define models in a clean, modular format.

```python
class MyNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyNeuralNetwork, self).__init__()


        # 각 레이어들을 nn.Sequential을 이용해서 순차적으로 쌓는다. ( 여기에서는, 레이어1은 conv -> ReLU -> Maxpool 순으로 쌓아진다.)
        # 이렇게 하면, 깊은 DNN 을 만들때 편하게 만들 수 있다.

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(in_features=30*5*5, out_features=128, bias=True),
            nn.ReLU(inplace=True)
        )

        self.layer4 = nn.Sequential(
            nn.Linear(in_features=128, out_features=10, bias=True),
            nn.ReLU(inplace=True)
        )


    # 각 레이어를 만들때, Sequential을 통해서 만들었다.
    # forward 연산을 통해서 NN의 구조를 형성할때, 보다 간편하고 명시적으로 쌓을 수 있도록 한다.
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.layer3(x)
        x = self.layer4(x)

        return x
```



In [1]:
# Sequential을 통해서, 레이어를 순차적으로 쌓아서 NN 모델을 쉽게 구현한다.
# 이 모델은, 입력노드2개 출력노드1개 인 NN에서 마지막 Activation Function으로 Sigmoid를 사용한 모델이다.

model = nn.Sequential(
   nn.Linear(2, 1), # input_dim = 2, output_dim = 1
   nn.Sigmoid() # 출력은 시그모이드 함수를 거친다
)



model(x_train)

NameError: name 'nn' is not defined

여기에서 model이라는 nn.Sequential 클래스의 인스턴스를 작성한다.

이 model의 인스턴스에 x_train데이터를 전달하면,  이 데이터는 
nn.Sequential에 정의된 forward method를 통해서 자동으로 아까 정의한 레이어( nn.Linear ,nn.Sigmoid )들로 보내진다.

( 이 forward 메소드는 model이 선언될때, 자동으로 실행된다. 
상속받은 nn.Module 클래스에는 __call__ 메소드가 있는데 얘가 인스턴스가 받는 인수를 자동으로 forward로 전달해준다.)

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주

        # 여기서 prediction은 torch.BoolTensor이다. 즉, tensor형태로 저장된 hypothesis를 >= 를 통해
        # element-wise로 비교한 결과를 가지고 있다.

        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주

        # 여기서, boolTensor를 floatTensor로 변환해야, y_train과 비교가 가능하다.


        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산

        # 먼저 correct_prediction에는 0,1이 담긴 텐서이다. 여기서 sum()을 통해 1의 개수를 셀 수 있다.
        # sum()을 통해서, 텐서 원소의 합이 담긴 1차원 텐서가 만들어진다. 이를 item()을 통해 파이썬 스칼라값을 꺼낸다.
        # len(correct_prediction)은 전체 원소의 개수를 나타낸다. 이를 통해 정확도를 계산한다.

        
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))


Epoch    0/1000 Cost: 0.539713 Accuracy 83.33%
Epoch   10/1000 Cost: 0.614853 Accuracy 66.67%


Epoch   20/1000 Cost: 0.441875 Accuracy 66.67%
Epoch   30/1000 Cost: 0.373145 Accuracy 83.33%
Epoch   40/1000 Cost: 0.316358 Accuracy 83.33%
Epoch   50/1000 Cost: 0.266094 Accuracy 83.33%
Epoch   60/1000 Cost: 0.220498 Accuracy 100.00%
Epoch   70/1000 Cost: 0.182095 Accuracy 100.00%
Epoch   80/1000 Cost: 0.157299 Accuracy 100.00%
Epoch   90/1000 Cost: 0.144091 Accuracy 100.00%
Epoch  100/1000 Cost: 0.134272 Accuracy 100.00%
Epoch  110/1000 Cost: 0.125769 Accuracy 100.00%
Epoch  120/1000 Cost: 0.118297 Accuracy 100.00%
Epoch  130/1000 Cost: 0.111680 Accuracy 100.00%
Epoch  140/1000 Cost: 0.105779 Accuracy 100.00%
Epoch  150/1000 Cost: 0.100483 Accuracy 100.00%
Epoch  160/1000 Cost: 0.095704 Accuracy 100.00%
Epoch  170/1000 Cost: 0.091369 Accuracy 100.00%
Epoch  180/1000 Cost: 0.087420 Accuracy 100.00%
Epoch  190/1000 Cost: 0.083806 Accuracy 100.00%
Epoch  200/1000 Cost: 0.080486 Accuracy 100.00%
Epoch  210/1000 Cost: 0.077425 Accuracy 100.00%
Epoch  220/1000 Cost: 0.074595 Accuracy 100.

### 클래스를 이용해서, 나만의 모델 구현 형식을 만들어보기

실제 머신러닝 혹은 신경망을 구성할때에는,  여러가지 알고리즘이나 회귀등을 내 입맛대로
추가하거나 빼서 설계해야한다.

이때, torch로부터 nn.Module에 정의된 여러가지 회귀나 함수들을 불러와야 하는데 이때 이렇게 하는 방식을 익혀두어야 한다.

In [24]:
class BinaryClassifier(nn.Module):
    def __init__(self):

        # super를 통해서, nn.Module 클래스의 속성(attribute)들을 상속받는다.
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()


    # forward 함수는 모델이 학습데이터를 입력받아서 forward 연산을 진행시키는 함수이다.

    # foward 메소드는 인스턴스가 호출될때, 자동으로 실행된다. 따라서, 모델을 인스턴스화하고,
    # 인스턴스에 입력값을 넣으면 자동으로 forward 연산이 진행된다.
    # 이는 매우 중요한데, 그 이유는 모델이 학습데이터를 입력받아서 예측값을 내놓는 과정을 정의하기 때문이다.
    def forward(self, x):
        return self.sigmoid(self.linear(x))
    

    # 여기서 볼 수  있듯, forward 연산은 우리의 모델인 H(x)=sigmoid(Wx+b)를 정의하고 있다.


# model을 생성하고, model에 x_train을 넣어서 forward 연산을 진행한다.
model = BinaryClassifier()


# optimizer 설정 ; 확률적 경사하강법을 사용한다.
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산, x_train을 넣어서 forward 연산을 진행한다.
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))


Epoch    0/1000 Cost: 0.614994 Accuracy 66.67%
Epoch   10/1000 Cost: 0.747550 Accuracy 83.33%
Epoch   20/1000 Cost: 0.633216 Accuracy 83.33%
Epoch   30/1000 Cost: 0.538123 Accuracy 83.33%
Epoch   40/1000 Cost: 0.450406 Accuracy 83.33%
Epoch   50/1000 Cost: 0.366382 Accuracy 83.33%
Epoch   60/1000 Cost: 0.287368 Accuracy 83.33%
Epoch   70/1000 Cost: 0.219289 Accuracy 83.33%
Epoch   80/1000 Cost: 0.173225 Accuracy 100.00%
Epoch   90/1000 Cost: 0.151674 Accuracy 100.00%
Epoch  100/1000 Cost: 0.140280 Accuracy 100.00%
Epoch  110/1000 Cost: 0.131002 Accuracy 100.00%
Epoch  120/1000 Cost: 0.122903 Accuracy 100.00%
Epoch  130/1000 Cost: 0.115765 Accuracy 100.00%
Epoch  140/1000 Cost: 0.109426 Accuracy 100.00%
Epoch  150/1000 Cost: 0.103760 Accuracy 100.00%
Epoch  160/1000 Cost: 0.098664 Accuracy 100.00%
Epoch  170/1000 Cost: 0.094056 Accuracy 100.00%
Epoch  180/1000 Cost: 0.089870 Accuracy 100.00%
Epoch  190/1000 Cost: 0.086050 Accuracy 100.00%
Epoch  200/1000 Cost: 0.082549 Accuracy 100.00%
